### Input files: Price-Today, orders-log
### Output files: Price-for-Order, orders-log, monthly-sales

In [2]:
import glob
import os
import pandas as pd
from datetime import date, timedelta, datetime
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'cost':'{:,.2f}','price':'{:,.2f}','%change':'{:,.2f}','%profit':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'cst_amt':'{:,.2f}','mkt_amt':'{:,.2f}','difa':'{:,.2f}',  
              'pct':'{:,.2f}','net':'{:,.2f}'}

pd.options.display.max_rows = 50
pd.options.display.float_format = '{:.2f}'.format

today = date.today()
yesterday = today - timedelta(days=1)
print(today, yesterday)

2025-08-04 2025-08-03


In [3]:
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2025-08-04
yesterday: 2025-08-01


In [4]:
# If run after actual work day
#today = today - timedelta(days=2)
print(today)

2025-08-04


In [5]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-08-04 20:21:26


### Restart and Run All Cells

In [7]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [8]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


## Must rerun whenever there is change in orders

In [10]:
sql = """
SELECT trade, name, qty, price AS target, active, reason, market, xdate 
FROM orders
ORDER BY trade, name"""
orders = pd.read_sql(sql, conlite)
orders.shape

(11, 8)

In [11]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [12]:
prices = pd.read_csv(input_file)
prices.shape

(169, 10)

In [13]:
df_merge  = pd.merge(orders, prices, on = 'name', how='inner')
df_merge.shape

(11, 17)

In [14]:
df_merge['spd'] = 0

In [15]:
colo = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'price', 'change', '%change',
        'active', 'xdate']
df_merge[colo].shape

(11, 12)

In [16]:
file_name = "Price-for-Order.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [17]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\Price-for-Order.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\Price-for-Order.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\Price-for-Order.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\Price-for-Order.csv


In [18]:
df_merge[colo].to_csv(output_file, header=True, index=False)
df_merge[colo].to_csv(god_file, header=True, index=False)
df_merge[colo].to_csv(icd_file, header=True, index=False)
df_merge[colo].to_csv(osd_file, header=True, index=False)

### call ruby ruby\order-log-new.rb

In [20]:
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'

In [21]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [22]:
!ruby ruby\\order-log-new.rb

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
B,DIF,-10,RD40Pct,SET,5000,7.7,8.2,0.05,0.613497,1,2025-05-14
B,GVREIT,-7,RND20Pct,SET,5000,6.2,6.55,0.0,0.0,1,2025-05-28
B,JMT,-25,RD75pct,SET50,2500,9.0,10.5,-0.1,-0.943396,0,2025-02-25
B,SENA,-30,RD70pct,SET,15000,1.44,1.74,-0.03,-1.694915,0,2025-05-09
B,WHAIR,-44,RD50pct,SET,10000,4.3,5.45,-0.05,-0.909091,0,2025-05-26
B,WHART,-24,RD30Pct,SET,5000,8.0,9.2,-0.1,-1.075269,0,2025-05-19
S,AIMIRT,18,RND15pct,SET,2500,11.5,9.8,-0.05,-0.507614,0,2025-05-19
S,IVL,13,CSSTGT,SET50,1000,24.0,22.7,0.3,1.339286,0,2025-05-27
S,MCS,8,20Pct,SET,6000,8.75,8.35,0.05,0.60241,1,2025-04-18
S,NER,4,CSSMIN,SET,5000,4.5,4.42,-0.04,-0.896861,1,2025-04-24
S,PTG,83,COS,SET100,3600,11.4,6.55,0.0,0.0,0,2025-03-10


In [23]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'

In [24]:
file_name   = 'orders-log.csv'
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\orders-log.csv


In [25]:
df = pd.read_csv(input_file, sep=',', index_col=None)
df['amount'] = df['qty'] * df['target']
df.query('spd >= -3 and spd <= 3')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [26]:
df.query('active == 2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [27]:
df.query('active == 1')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,DIF,-10,RD40Pct,SET,5000,7.70,8.20,0.05,0.61,1,2025-05-14,38500.00
1,B,GVREIT,-7,RND20Pct,SET,5000,6.20,6.55,0.00,0.00,1,2025-05-28,31000.00
8,S,MCS,8,20Pct,SET,6000,8.75,8.35,0.05,0.60,1,2025-04-18,52500.00
9,S,NER,4,CSSMIN,SET,5000,4.50,4.42,-0.04,-0.90,1,2025-04-24,22500.00


In [28]:
df.query('active == 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
2,B,JMT,-25,RD75pct,SET50,2500,9.00,10.50,-0.10,-0.94,0,2025-02-25,22500.00
3,B,SENA,-30,RD70pct,SET,15000,1.44,1.74,-0.03,-1.69,0,2025-05-09,21600.00
4,B,WHAIR,-44,RD50pct,SET,10000,4.30,5.45,-0.05,-0.91,0,2025-05-26,43000.00
5,B,WHART,-24,RD30Pct,SET,5000,8.00,9.20,-0.10,-1.08,0,2025-05-19,40000.00
6,S,AIMIRT,18,RND15pct,SET,2500,11.50,9.80,-0.05,-0.51,0,2025-05-19,28750.00
7,S,IVL,13,CSSTGT,SET50,1000,24.00,22.70,0.30,1.34,0,2025-05-27,24000.00
10,S,PTG,83,COS,SET100,3600,11.40,6.55,0.00,0.00,0,2025-03-10,41040.00


In [29]:
df.query('spd <= -40 and trade == "B"')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
4,B,WHAIR,-44,RD50pct,SET,10000,4.30,5.45,-0.05,-0.91,0,2025-05-26,43000.00


In [30]:
df.query('trade == "B" & spd > 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [31]:
df.query('trade == "S" & spd < 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [32]:
colx = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'current', 'change', '%change',
        'active', 'xdate']

In [33]:
file_name = "orders-log.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [34]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\orders-log.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\orders-log.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\orders-log.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\orders-log.csv


In [35]:
df[colx].to_csv(output_file, header=True, index=False)
df[colx].to_csv(god_file, header=True, index=False)
df[colx].to_csv(icd_file, header=True, index=False)
df[colx].to_csv(osd_file, header=True, index=False)

### Convert column name to make it valid for df.query

In [37]:
df['percent'] = df['%change']

In [38]:
df.query('percent < -2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent


In [39]:
df.query('percent >= 2.00')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent


In [40]:
df.loc[df.trade == 'B'].sort_values('spd',ascending=False)

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
1,B,GVREIT,-7,RND20Pct,SET,5000,6.20,6.55,0.00,0.00,1,2025-05-28,31000.00,0.00
0,B,DIF,-10,RD40Pct,SET,5000,7.70,8.20,0.05,0.61,1,2025-05-14,38500.00,0.61
5,B,WHART,-24,RD30Pct,SET,5000,8.00,9.20,-0.10,-1.08,0,2025-05-19,40000.00,-1.08
2,B,JMT,-25,RD75pct,SET50,2500,9.00,10.50,-0.10,-0.94,0,2025-02-25,22500.00,-0.94
3,B,SENA,-30,RD70pct,SET,15000,1.44,1.74,-0.03,-1.69,0,2025-05-09,21600.00,-1.69
4,B,WHAIR,-44,RD50pct,SET,10000,4.30,5.45,-0.05,-0.91,0,2025-05-26,43000.00,-0.91


### Sell recently purchased stocks if profitable

In [42]:
#pd.set_option('display.max_rows', 40)
sql = """SELECT name, qty, price AS cost, date 
FROM buys B 
JOIN stocks S 
ON stock_id = S.id 
WHERE status = 'Active'
ORDER BY name, date"""
buys = pd.read_sql(sql, conpf)
buys.shape

(111, 4)

In [43]:
colu = 'name date_x qty cost price difa %change %profit cst_amt mkt_amt'.split()
buys_price = pd.merge(buys, prices, on='name', how='inner')
buys_price['diff'] = buys_price['price'] - buys_price['cost']
buys_price['difa'] = buys_price['diff'] * buys_price['qty']
buys_price['%profit'] = round(buys_price['difa'] / (buys_price['qty'] * buys_price['cost']) * 100,2)

In [44]:
buys_price['cst_amt'] = buys_price['qty'] * buys_price['cost']
buys_price['mkt_amt'] = buys_price['qty'] * buys_price['price']

In [45]:
laggards = buys_price.loc[buys_price.cost > buys_price.price,colu]
laggards.shape

(104, 10)

In [46]:
watch_list = buys_price.loc[buys_price.cost <= buys_price.price,colu]
watch_list.sort_values('%profit',ascending=False).style.format(format_dict)

,name,date_x,qty,cost,price,difa,%change,%profit,cst_amt,mkt_amt
51,MCS,2025-02-21,"6,000",7.30,8.35,"6,300.00",0.60,14.38,"43,800.00","50,100.00"
56,NER,2025-06-23,"5,000",3.90,4.42,"2,600.00",-0.90,13.33,"19,500.00","22,100.00"
21,DIF,2025-04-09,"5,000",7.50,8.20,"3,500.00",0.61,9.33,"37,500.00","41,000.00"
31,GVREIT,2023-09-22,"6,000",6.10,6.55,"2,700.00",0.00,7.38,"36,600.00","39,300.00"
110,WHART,2025-07-08,"5,000",8.80,9.20,"2,000.00",-1.08,4.55,"44,000.00","46,000.00"
74,RCL,2022-10-20,"3,000",27.75,28.50,"2,250.00",-2.56,2.70,"83,250.00","85,500.00"
42,JMT,2025-05-19,"2,800",10.50,10.50,0.00,-0.94,0.00,"29,400.00","29,400.00"


In [47]:
float_formatter = "฿{:,.2f}"
float_formatter.format(watch_list.difa.sum())

'฿19,350.00'

In [48]:
sell_list = buys_price.loc[buys_price['%profit'] >= 10.00,colu]
sell_list.sort_values('%profit',ascending=False).style.format(format_dict)

,name,date_x,qty,cost,price,difa,%change,%profit,cst_amt,mkt_amt
51,MCS,2025-02-21,"6,000",7.30,8.35,"6,300.00",0.60,14.38,"43,800.00","50,100.00"
56,NER,2025-06-23,"5,000",3.90,4.42,"2,600.00",-0.90,13.33,"19,500.00","22,100.00"


In [49]:
float_formatter = "฿{:,.2f}"
float_formatter.format(sell_list.cst_amt.sum())

'฿63,300.00'

In [50]:
float_formatter = "฿{:,.2f}"
float_formatter.format(sell_list.mkt_amt.sum())

'฿72,200.00'

In [51]:
float_formatter = "฿{:,.2f}"
float_formatter.format(sell_list.difa.sum())

'฿8,900.00'

### Profit per Month

In [53]:
year = 2025
month = 8
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2025 AND MONTH(sells.date) = 8
ORDER BY sells.date DESC, name


In [54]:
sells_df

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days


In [55]:
print(f"Monthly Profit Amount: {sells_df.gross.sum():,.2f}")

Monthly Profit Amount: 0.00


In [56]:
file_name = "monthly-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [57]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\monthly-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\monthly-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\monthly-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\monthly-sales.csv


In [58]:
sells_df.to_csv(output_file, header=True, index=False)
sells_df.to_csv(god_file, header=True, index=False)
sells_df.to_csv(icd_file, header=True, index=False)
sells_df.to_csv(osd_file, header=True, index=False)

In [59]:
sells_df['profit_per_day'] = (sells_df['gross'] / sells_df['buy_amt']) / sells_df['days'] * 100
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days,profit_per_day


In [60]:
float_formatter = "฿{:,.2f}"
float_formatter.format(sells_df.gross.sum())

'฿0.00'

In [61]:
average_profit_percent = sells_df['profit_per_day'].mean()
print(f"Average Profit Percent: {average_profit_percent:.2f}%")

Average Profit Percent: nan%


In [62]:
#dt = date(2023, 11, 8)
dt = today
dt.strftime("%B %d, %Y")

'August 04, 2025'

In [63]:
print("Hello, {}! Today is {:%B %d, %Y}.".format("Alice", dt))

Hello, Alice! Today is August 04, 2025.


In [64]:
conlite.commit()
conlite.close()

In [65]:
# Get the current time
current_time = datetime.now()
# Format the time to remove millisAeconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-08-04 20:21:30
